# Face Transformer Analysis

In [ ]:
import numpy as np
import pandas as pd
import respiration.utils as utils

evaluation_dir = utils.dir_path('outputs', 'signals')

predictions_file = utils.join_paths(evaluation_dir, 'transformer_predictions.csv')
predictions = pd.read_csv(predictions_file)
predictions['signal'] = predictions['signal'].apply(eval).apply(np.array)
predictions.head()

## Compare the predictions to the ground truth

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()

In [ ]:
import respiration.analysis as analysis

models = predictions['model'].unique()

analysis_results = []

for model in models:
    signals = predictions[predictions['model'] == model]

    experiment_analysis = analysis.Analysis()

    for idx, row in signals.iterrows():
        subject, setting = row['subject'], row['setting']
        prediction = row['signal']
        gt_signal = dataset.get_breathing_signal(subject, setting)
        experiment_analysis.add_data(prediction, gt_signal, 30)

    metrics = experiment_analysis.compute_metrics()
    for entry in metrics:
        analysis_results.append({
            'model': model,
            'metric': entry['metric'],
            'method': entry['method'],
            'value': entry['value'],
        })

analysis_results = pd.DataFrame(analysis_results)

In [ ]:
analysis_results

In [ ]:
# Make all metrics positive
analysis_results['value'] = analysis_results['value'].abs()

# Invert the correlation metric --> Lower is now better
for idx, row in analysis_results[analysis_results['metric'] == 'Correlation'].iterrows():
    analysis_results.at[idx, 'value'] = 1 - row['value']

In [ ]:
analysis_results

In [ ]:
# Rank the models based on the metrics
model_rankings = []

for metric in analysis_results['metric'].unique():
    for method in analysis_results['method'].unique():
        metric_results = analysis_results[
            (analysis_results['metric'] == metric) &
            (analysis_results['method'] == method)]

        ranks = metric_results.groupby('model')['value'].mean().sort_values().index
        for idx, rank in enumerate(ranks):
            model_rankings.append({
                'model': rank,
                'metric': metric,
                'method': method,
                'rank': idx + 1
            })

model_rankings = pd.DataFrame(model_rankings)

In [ ]:
model_rankings

In [ ]:
# Only keep the MAE and Correlation metrics
model_rankings = model_rankings[model_rankings['metric'].isin(['MAE', 'Correlation'])]

In [ ]:
# Average the ranks for each model
average_ranks = model_rankings.groupby('model')['rank'].mean().sort_values().reset_index()
average_std = model_rankings.groupby('model')['rank'].std().sort_values().reset_index()

# Merge the average ranks and standard deviations
average_ranks = average_ranks.merge(average_std, on='model', suffixes=('_mean', '_std'))
average_ranks

## Show some predictions

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()

In [ ]:
import matplotlib.pyplot as plt

subject = 'Proband25'
setting = '303_normalized_face'

gt_signal = dataset.get_breathing_signal(subject, setting)
# gt_signal = analysis.butterworth_filter(gt_signal, 30, 0.08, 0.5)
gt_signal = analysis.normalize_signal(gt_signal)

prediction = predictions[
    (predictions['model'] == '20240710_142159') &
    (predictions['subject'] == subject) &
    (predictions['setting'] == setting)]['signal'].values[0]
# prediction = analysis.butterworth_filter(prediction, 30, 0.08, 0.5)
prediction = analysis.normalize_signal(prediction)

plt.figure(figsize=(20, 5))
plt.plot(gt_signal, label='Ground Truth')
plt.plot(prediction, label='Prediction')
plt.legend()
plt.show()

In [ ]:
# Plot all models for a single subject

plt.figure(figsize=(20, 5))

for model in models:
    prediction = predictions[
        (predictions['model'] == model) &
        (predictions['subject'] == subject) &
        (predictions['setting'] == setting)]['signal'].values[0]
    prediction = analysis.normalize_signal(prediction)
    plt.plot(prediction, label=model)

plt.title('Predictions for Proband21')
plt.legend()
plt.show()